In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.chdir('/Users/colerichmond/Documents/GitHub/DSC180B-Capstone')
!pwd

/Users/colerichmond/Documents/GitHub/DSC180B-Capstone


In [5]:
!python run.py data

Ingesting data...
...done!
Cleaning data...
Traceback (most recent call last):
  File "/Users/colerichmond/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2897, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 107, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1607, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'date'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "run.py", line 55, in <module>
    main(targets)
  File "run.py", line 38, in main
    clean_stops(**cfg)
  File "src/etl.py", line 213, in clean_stops
    cleaned = get_clean_stops(pth, df)
  File "src/etl.py", line 47, in get_clean_stops

In [ ]:
pd.read_csv('https://stacks.stanford.edu/file/druid:kx738rc7407/kx738rc7407_fl_statewide_2019_12_17.csv.zip', nrows=100)

In [7]:
"sc_cols" = ["county_name",
             "subject_age"
   ]
"fl_cols" = [
      "county_name",
      "officer_sex",
      "officer_years_of_service",
      "subject_age",
      "time",
      "vehicle_registration_state"
   ]
"pa_cols" = [
      "time",
      "lat",
      "lng",
      "officer_sex",
      "contraband_found"
   ]

SyntaxError: can't assign to literal (<ipython-input-7-8dacc9c85351>, line 1)

In [ ]:
all_cols = [
      "subject_race",
      "subject_sex",
      "officer_race",
      "officer_age",
      "violation",
      "arrest_made",
      "citation_issued",
      "search_conducted"
   ]

In [ ]:
def get_stops_url(location, all_cols, sc_cols, fl_cols, pa_cols):
    '''
    return a downloadable file of traffic
    stops for a given year before RIPA.
    
    :param: given year to fetch
    '''
    
    if location is "pa":
        url = 'https://stacks.stanford.edu/file/druid:yg821jf8611/yg821jf8611_pa_pittsburgh_2020_04_01.csv.zip'
        cols = all_cols + pa_cols
    else:
        url = 'https://stacks.stanford.edu/file/druid:kx738rc7407/kx738rc7407_%s_statewide_2019_12_17.csv.zip' % (location)
        if location is "sc":  
            cols = all_cols + sc_cols
        if location is "fl":
            cols = all_cols + fl_cols

    return pd.read_csv(url, nrows=100).loc[:, cols]

In [ ]:
for location in ["pa", "fl", "sc"]:
    table = get_stops_url(location, all_cols)
    file_name = '%s_stops.csv' % (location)
    table.to_csv(os.path.join("data/raw", file_name))